In [4]:
import os
import pandas as pd

HORNSGATAN_HOME = os.environ["HORNSGATAN_HOME"]
os.chdir(HORNSGATAN_HOME)
print("Current directory:", os.getcwd())

from src.tools import mytools


date = "2020-01-01"
detector = "e2w_in"
path  = "data/daily_splitted_data/"


Current directory: /home/kaveh/projects/Hornsgatan


In [5]:
data = pd.read_csv(f"{path}data_{date}.csv")
data.head()

,detector_id,time_detector_real,speed_detector_real,day,date
0,w2e_in,1577836811,33,Wednesday,2020-01-01
1,w2e_in,1577836814,27,Wednesday,2020-01-01
2,w2e_in,1577836815,32,Wednesday,2020-01-01
3,e2w_in,1577836818,32,Wednesday,2020-01-01
4,e2w_in,1577836820,27,Wednesday,2020-01-01


In [6]:
data = data[data["detector_id"]==detector]
data.head()

,detector_id,time_detector_real,speed_detector_real,day,date
3,e2w_in,1577836818,32,Wednesday,2020-01-01
4,e2w_in,1577836820,27,Wednesday,2020-01-01
6,e2w_in,1577836828,37,Wednesday,2020-01-01
7,e2w_in,1577836838,48,Wednesday,2020-01-01
9,e2w_in,1577836844,37,Wednesday,2020-01-01


In [7]:
len(data)

10747

In [8]:
from src.tools import mytools


In [9]:
#mytools.delete_all_except_list(f"data/sim_intermediate_data/", [".gitkeep"])